# Load the data from a chSPSIM acquisition

In [1]:
#import functions
import sys
import os
import glob
import numpy as np
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display


#import local functions
sys.path.append('/home/crombez/Documents/PhD/python/MOPDC/fonction/') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

In [2]:
%matplotlib 
#inline 

Using matplotlib backend: Qt5Agg


In [3]:
file_path_data = '/run/media/crombez/Elements/HSPIM_seb_acquisition/list_path_data'

with open(file_path_data, 'r') as f:

    list_path_data_and_info = [line.strip('\n') for line in f.readlines()]


list_path_data = []
list_path_info = []

    
for x in list_path_data_and_info :
    x_strip = x.split()
    list_path_data.append(x_strip[0])
    list_path_info.append(x_strip[1])
    
    
#n = len(list_path_data)
#print("****  Emplacent des dossiers contenant les jeux de données  **** \n ")
#for i in range(n):
#    print(str(i+1)+' ===> '+list_path_data[i]+' ===> '+list_path_info[i])
#    n = len(list_path_data[i])
#    Run = [x[0][n:] for x in os.walk(list_path_data[i])][1:]
#    print('Nom des sous-répertoires : ')
#    print(str(Run)+'\n')

In [4]:
def Select_data(Dir, Run):
    Data_path = Dir+Run+'/'
    name_type = os.listdir(Data_path)[0]
    name_type = name_type[:-9]+'*'+name_type[-4:]
    files = Files_names(Data_path,str(name_type))
    print('Données du dossier : '+Data_path)
    with open(Dir+'info_run',"r") as fichier:
        print(fichier.read())
    return(Data_path,files)

def change_set_acqui(*args):
    n = len(set_acqui.value)
    run.options = [x[0][n:] for x in os.walk(set_acqui.value)][1:]

In [5]:
n = len(list_path_data[0])
print(list_path_data[0])
Run = [x[0][n:] for x in os.walk(list_path_data[0])][1:]

style = {'description_width': 'initial'}
set_acqui = widgets.Dropdown(options=list_path_data, value=list_path_data[0], description='Set of data', style = style)    
run = widgets.Dropdown(options=Run, description='Run', style = style)

set_acqui.observe(change_set_acqui, 'value')  



Data1 = interactive(Select_data, Dir = set_acqui,
                    Run = run)
display(Data1)

/run/media/crombez/Elements/HSPIM_Pierre_acquisition/Convert/


interactive(children=(Dropdown(description='Set of data', options=('/run/media/crombez/Elements/HSPIM_Pierre_a…

In [ ]:
Data_path1, Files1 = Data1.result
N_run = Data_path1[-2]
print('Données du dossier : '+Data_path1)
print('Nombre de fichier : '+str(len(Files1)))
print('Run n° : '+str(N_run))

In [ ]:
# Hyper cube dimensions
Nl = 2160 # number of pixcels along the y dimensions 
Nc = 128 # number of measured Walsh_Hadmard coefficients (correpond to the h dimensions)
Nd = 2560 # number of pixcels along the spectral dimensions 

In [ ]:
%%time
Raw_data = load_hyper_cube(Data_path1,Files1,Nl,Nc,Nd)

#### Check data by looking at the lambda-stack of the hyper cube

In [ ]:
W = walsh_ordered(2048)
stack = np.zeros((Nl,2048))
for i in range(Nd):
    stack += np.dot(Raw_data[:,:,i],W[:Nc])
plot_im2D(stack,cmap='gray')

#### Save the data

In [ ]:
Save_path = "/home/crombez/Documents/PhD/python/MOPDC/Data_save/Raw_data/"
file_name = "raw_rhodamin_2160x128x2560.npy"
np.save(Save_path+file_name,Raw_data)